In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib


In [2]:
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url, sep=';')

print(data.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [3]:
print (data.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

In [4]:
y = data.quality
X = data.drop('quality', axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)

In [6]:
scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)


In [7]:
print X_train_scaled.mean(axis=0)
print X_train_scaled.std(axis=0)

[ 1.16664562e-16 -3.05550043e-17 -8.47206937e-17 -2.22218213e-17
  2.22218213e-17 -6.38877362e-17 -4.16659149e-18 -2.54439854e-15
 -8.70817622e-16 -4.08325966e-16 -1.17220107e-15]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [8]:
X_test_scaled = scaler.transform(X_test)
print X_train_scaled.mean(axis=0)
print X_train_scaled.std(axis=0)


pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

print pipeline.get_params()

hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

[ 1.16664562e-16 -3.05550043e-17 -8.47206937e-17 -2.22218213e-17
  2.22218213e-17 -6.38877362e-17 -4.16659149e-18 -2.54439854e-15
 -8.70817622e-16 -4.08325966e-16 -1.17220107e-15]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
{'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__max_leaf_nodes': None, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__min_impurity_split': None, 'standardscaler__copy': True, 'randomforestregressor__oob_score': False, 'randomforestregressor__min_weight_fraction_leaf': 0.0, 'randomforestregressor__min_samples_split': 2, 'memory': None, 'randomforestregressor__min_impurity_decrease': 0.0, 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor__random_state': None, 'standardscaler__with_mean': True, 'randomforestregressor__criterion': 'mse', 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,


In [9]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)

clf.fit(X_train,y_train)
print clf.best_params_
print clf.refit


{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'auto'}
True


In [17]:
 	
y_pred = clf.predict(X_test)
print r2_score(y_test, y_pred)
# 0.45044082571584243
 
print mean_squared_error(y_test, y_pred)
# 0.35461593750000003

0.4615431926872088
0.347451875


In [20]:
joblib.dump(clf, 'rf_regressor.pkl')
clf2 = joblib.load('rf_regressor.pkl')
clf2.predict(X_test)

array([6.57, 5.81, 5.01, 5.61, 6.4 , 5.61, 4.76, 4.68, 5.03, 6.19, 5.24,
       5.7 , 5.78, 5.  , 5.75, 5.63, 6.64, 5.62, 5.74, 6.96, 5.46, 5.74,
       4.95, 6.03, 5.89, 5.08, 5.56, 5.16, 5.92, 5.93, 5.89, 6.61, 5.99,
       5.01, 4.92, 5.9 , 5.01, 5.78, 5.04, 5.76, 4.82, 5.95, 6.88, 5.06,
       6.27, 5.32, 5.6 , 5.49, 5.08, 6.56, 5.93, 5.28, 5.88, 5.19, 5.51,
       5.79, 5.26, 5.37, 5.  , 5.38, 5.25, 5.17, 5.04, 5.78, 5.97, 5.22,
       6.32, 5.03, 5.11, 6.52, 5.75, 5.36, 5.1 , 5.02, 5.24, 6.01, 5.22,
       5.14, 5.14, 5.24, 6.65, 5.43, 6.23, 6.51, 5.11, 5.85, 6.56, 6.02,
       5.54, 5.72, 5.91, 5.39, 6.6 , 5.71, 5.69, 5.8 , 6.67, 6.68, 5.49,
       6.88, 5.02, 5.32, 5.12, 6.61, 5.03, 4.71, 5.77, 5.13, 5.77, 5.98,
       5.69, 5.43, 6.09, 5.55, 5.01, 5.24, 5.84, 5.06, 4.78, 6.04, 5.91,
       5.13, 5.8 , 5.91, 5.24, 5.27, 5.26, 5.86, 5.47, 5.41, 5.72, 6.26,
       5.1 , 5.37, 5.04, 6.44, 5.01, 5.1 , 6.69, 5.52, 5.2 , 5.03, 5.83,
       6.1 , 5.34, 5.42, 5.14, 6.61, 5.69, 5.03, 5.